# Numerical Benchmarks

In [1]:
import sys
sys.path.insert(0, '../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle

from qiskit.quantum_info import DensityMatrix
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm

from loss_functions import *
from optimization import *
from quantum_maps import *
from quantum_tools import *
from experiments import *
#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=3)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

## Check Convergence

### Overparameterization

In [2]:
n = 3
d = 2**n
num_reps = 10
num_iter = 4000

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

inputs, _ = generate_pauli_circuits(n = n, 
                                    circuit_target=None, 
                                    N = None, 
                                    trace=False)

rank = 3
loss_values = np.zeros((num_iter, num_reps))
c_values = np.zeros((num_iter, num_reps))


for i in tqdm(range(num_reps)):
    X, _, _ = generate_ginibre(d, d)
    U = generate_unitary(X) # Random unitary part
    kraus_target = KrausMap(U, 0.5, d, rank)

    states = kraus_target.apply_map(inputs[0])
    targets = measurement(states, U_basis = inputs[1])
    
    c = random.uniform(0,1)
    kraus_model = KrausMap(
                       U=U, 
                       c=c, 
                       d=d, 
                       rank=d**2-1
                      )

    model = ModelQuantumMap(
                           q_map = kraus_model,
                           loss = probs_loss,
                           optimizer = tf.optimizers.Adam(learning_rate=0.01),
                           )


    model.train(inputs = inputs,
                targets = targets,
                inputs_val = None,
                targets_val = [kraus_target],
                num_iter = num_iter,
                N = 500,
                verbose = False,
                )
    
    loss_values[:, i] = np.array(model.loss_list)
    c_values[:, i] = np.array(model.c_list)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

...

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

In [3]:
data = [loss_values, c_values]
pickle.dump(data, open("../../data/compressed_sensing_overparam.p", "wb"))

### Under-parameterization

In [15]:
n = 3
d = 2**n
num_reps = 10
num_iter = 4000

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

inputs, _ = generate_pauli_circuits(n = n, 
                                    circuit_target=None, 
                                    N = None, 
                                    trace=False)

rank = 3
loss_values = np.zeros((num_iter, num_reps))
c_values = np.zeros((num_iter, num_reps))


for i in tqdm(range(num_reps)):
    X, _, _ = generate_ginibre(d, d)
    U = generate_unitary(X) # Random unitary part
    kraus_target = KrausMap(U, 0.5, d, rank=7)

    states = kraus_target.apply_map(inputs[0])
    targets = measurement(states, U_basis = inputs[1])
    
    c = random.uniform(0,1)
    kraus_model = KrausMap(
                       U=U, 
                       c=c, 
                       d=d, 
                       rank=rank
                      )

    model = ModelQuantumMap(
                           q_map = kraus_model,
                           loss = probs_loss,
                           optimizer = tf.optimizers.Adam(learning_rate=0.01),
                           )


    model.train(inputs = inputs,
                targets = targets,
                inputs_val = None,
                targets_val = [kraus_target],
                num_iter = num_iter,
                N = 500,
                verbose = False,
                )
    
    loss_values[:, i] = np.array(model.loss_list)
    c_values[:, i] = np.array(model.c_list)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
data = [loss_values, c_values]
pickle.dump(data, open("../../data/compressed_sensing_underparam.p", "wb"))

## Sparse Input
### Generate Data

In [6]:
np.random.seed(42)
random.seed(42)

N_list = [int(16*2**(i/4)) for i in range(21)]
num_reps = 10

### Low Rank

In [7]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

n = 3
d = 2**n
rank = 3

prior_fidelity = np.zeros((len(N_list), num_reps))
noPrior_fidelity = np.zeros((len(N_list), num_reps))

for rep in tqdm(range(num_reps)):
    X, _, _ = generate_ginibre(d, d)
    U = generate_unitary(X) # Random unitary part
    kraus_target = KrausMap(U, 0.5, d, rank)
    
    inputs, _ = generate_pauli_circuits(n = n, 
                                        circuit_target=None, 
                                        N = None, 
                                        trace=False)
    states = kraus_target.apply_map(inputs[0])
    targets = measurement(states, U_basis = inputs[1])
    
    for i, N in enumerate(N_list):
        kraus_model = KrausMap(
                               U=U, 
                               c=0.9, 
                               d=d, 
                               rank=rank,
                              )

        model = ModelQuantumMap(
                                q_map = kraus_model,
                                loss = probs_loss,
                                optimizer = tf.optimizers.Adam(learning_rate=0.01),
                                )

        model.train(inputs = [inputs[0][:N], inputs[1][:N]],
                    targets = targets[:N],
                    inputs_val = None,
                    targets_val = None,
                    num_iter = 4000,
                    N = N,
                    verbose = False,
                    )

        prior_fidelity[i, rep] = np.abs(channel_fidelity(kraus_target, kraus_model).numpy())

        kraus_model = KrausMap(
    #                           U=U, 
    #                           c=0.9, 
                               d=d, 
                               rank=rank+1,
                              )

        model = ModelQuantumMap(
                                q_map = kraus_model,
                                loss = probs_loss,
                                optimizer = tf.optimizers.Adam(learning_rate=0.01),
                                )

        model.train(inputs = [inputs[0][:N], inputs[1][:N]],
                    targets = targets[:N],
                    inputs_val = None,
                    targets_val = None,
                    num_iter = 4000,
                    N = N,
                    verbose = False,
                    )

        noPrior_fidelity[i, rep] = np.abs(channel_fidelity(kraus_target, kraus_model).numpy())

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

...

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

In [8]:
data = [prior_fidelity, noPrior_fidelity]
pickle.dump(data, open("../../data/compressed_sensing_rank4.p", "wb"))

### Medium Rank

In [9]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

n = 3
d = 2**n
rank = 7

prior_fidelity2 = np.zeros((len(N_list), num_reps))
noPrior_fidelity2 = np.zeros((len(N_list), num_reps))

for rep in tqdm(range(num_reps)):
    X, _, _ = generate_ginibre(d, d)
    U = generate_unitary(X) # Random unitary part
    kraus_target = KrausMap(U, 0.5, d, rank)
    
    inputs, _ = generate_pauli_circuits(n = n, 
                                        circuit_target=None, 
                                        N = None, 
                                        trace=False)
    
    states = kraus_target.apply_map(inputs[0])
    targets = measurement(states, U_basis = inputs[1])
    
    for i, N in enumerate(N_list):
        kraus_model = KrausMap(
                               U=U, 
                               c=0.9, 
                               d=d, 
                               rank=rank,
                              )

        model = ModelQuantumMap(
                                q_map = kraus_model,
                                loss = probs_loss,
                                optimizer = tf.optimizers.Adam(learning_rate=0.01),
                                )

        model.train(inputs = [inputs[0][:N], inputs[1][:N]],
                    targets = targets[:N],
                    inputs_val = None,
                    targets_val = None,
                    num_iter = 4000,
                    N = N,
                    verbose = False,
                    )

        prior_fidelity2[i, rep] = np.abs(channel_fidelity(kraus_target, kraus_model).numpy())

        kraus_model = KrausMap(
    #                           U=U, 
    #                           c=0.9, 
                               d=d, 
                               rank=rank+1,
                              )

        model = ModelQuantumMap(
                                q_map = kraus_model,
                                loss = probs_loss,
                                optimizer = tf.optimizers.Adam(learning_rate=0.01),
                                )

        model.train(inputs = [inputs[0][:N], inputs[1][:N]],
                    targets = targets[:N],
                    inputs_val = None,
                    targets_val = None,
                    num_iter = 4000,
                    N = N,
                    verbose = False,
                    )

        noPrior_fidelity2[i, rep] = np.abs(channel_fidelity(kraus_target, kraus_model).numpy())

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

...

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

In [10]:
data = [prior_fidelity2, noPrior_fidelity2]
pickle.dump(data, open("../../data/compressed_sensing_rank8.p", "wb"))

### High Rank

In [11]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

n = 3
d = 2**n
rank = 15

prior_fidelity3 = np.zeros((len(N_list), num_reps))
noPrior_fidelity3 = np.zeros((len(N_list), num_reps))

for rep in tqdm(range(num_reps)):
    X, _, _ = generate_ginibre(d, d)
    U = generate_unitary(X) # Random unitary part
    kraus_target = KrausMap(U, 0.5, d, rank)
    
    inputs, _ = generate_pauli_circuits(n = n, 
                                        circuit_target=None, 
                                        N = None, 
                                        trace=False)
    
    states = kraus_target.apply_map(inputs[0])
    targets = measurement(states, U_basis = inputs[1])
    
    for i, N in enumerate(N_list):
        kraus_model = KrausMap(
                               U=U, 
                               c=0.9, 
                               d=d, 
                               rank=rank,
                              )

        model = ModelQuantumMap(
                                q_map = kraus_model,
                                loss = probs_loss,
                                optimizer = tf.optimizers.Adam(learning_rate=0.01),
                                )

        model.train(inputs = [inputs[0][:N], inputs[1][:N]],
                    targets = targets[:N],
                    inputs_val = None,
                    targets_val = None,
                    num_iter = 4000,
                    N = N,
                    verbose = False,
                    )

        prior_fidelity3[i, rep] = np.abs(channel_fidelity(kraus_target, kraus_model).numpy())

        kraus_model = KrausMap(
    #                           U=U, 
    #                           c=0.9, 
                               d=d, 
                               rank=rank+1,
                              )

        model = ModelQuantumMap(
                                q_map = kraus_model,
                                loss = probs_loss,
                                optimizer = tf.optimizers.Adam(learning_rate=0.01),
                                )

        model.train(inputs = [inputs[0][:N], inputs[1][:N]],
                    targets = targets[:N],
                    inputs_val = None,
                    targets_val = None,
                    num_iter = 4000,
                    N = N,
                    verbose = False,
                    )

        noPrior_fidelity3[i, rep] = np.abs(channel_fidelity(kraus_target, kraus_model).numpy())

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

...

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

In [ ]:
data = [prior_fidelity3, noPrior_fidelity3]
pickle.dump(data, open("../../data/compressed_sensing_rank16.p", "wb"))

## Collapse of SPAM optimization

In [47]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

n = 2
d = 2**n
rank = d

spam_target = SPAM(d=d, 
                   init = init_ideal(d),
                   povm = povm_ideal(d),
                   )

### Generate Synthetic Data

In [48]:
inputs_spam, _ = generate_pauliInput_circuits(n)

U_prep = inputs_spam
N_spam = U_prep.shape[0]

state = tf.repeat(spam_target.init[None,:,:], N_spam, axis=0)
state = apply_unitary(state, U_prep)
targets_spam = measurement(state, povm = spam_target.povm)

### Fit SPAM model, no Pre-Training

In [49]:
np.random.seed(43)
random.seed(43)
tf.random.set_seed(43)

spam_model = SPAM(d=d,
                  optimizer = tf.optimizers.Adam(learning_rate=0.01))

spam_model.train(inputs = inputs_spam,
                 targets = targets_spam,
                 num_iter = 1000,
                 N = N_spam
                 )

  0%|          | 0/1000 [00:00<?, ?it/s]

0 0.8921902649442512
1 0.8791598613811485
2 0.8664378444021045
3 0.8540343076056149
4 0.8419629923787568
5 0.8302745546766235
6 0.8189254316461284
7 0.8079260217838801
8 0.7973288674532693
9 0.7870964791473019
10 0.7772524672816913
11 0.7678012329386679
12 0.7587422764817371
13 0.7500702533673007
14 0.7417746797010674
15 0.7338403029673441
16 0.7262488917928424
17 0.7189027999203574
18 0.7119036809497684
19 0.705160706150451
20 0.698659058733764
21 0.692371500136835
22 0.6862699551170234
23 0.6803124724279443
24 0.6744751479178338
25 0.6685125168635269
26 0.662551184718625
27 0.6565805147614105
28 0.650576358048396
29 0.6445173936169588
30 0.6383315697365326
31 0.6320002069949816
32 0.625558948489927
33 0.6190020285950991
34 0.6123305431961068
35 0.6055321927646856
36 0.5985269314117968
37 0.5914369933423578
38 0.5843542213137325
39 0.5772045325834528
40 0.5699881035740306
41 0.5626968709832004
42 0.5553200487824677
43 0.5479018421513427
44 0.5406513321294362
45 0.5333543059059596
46 0

355 0.001475376542439607
356 0.0014602384653485745
357 0.0014453134123963449
358 0.0014305977308113255
359 0.0014160878207448383
360 0.001401780130375531
361 0.001387671152002321
362 0.001373757419233668
363 0.0013600355053461368
364 0.0013465020228342129
365 0.0013331536241198841
366 0.0013199870033331444
367 0.0013069988990203841
368 0.0012941860975887124
369 0.001281545437263897
370 0.0012690738123173266
371 0.00125676817731289
372 0.0012446255511396334
373 0.0012326430206174238
374 0.0012208177435044164
375 0.0012091469507726138
376 0.0011976279480742516
377 0.0011862581163608185
378 0.001175034911664172
379 0.0011639558640857428
380 0.0011530185760675324
381 0.001142220720039128
382 0.0011315600355436613
383 0.0011210343259504146
384 0.0011106414548597995
385 0.0011003793422911636
386 0.0010902459607477334
387 0.0010802393312268114
388 0.0010703575192484707
389 0.001060598630958376
390 0.0010509608093635134
391 0.0010414422307503748
392 0.0010320411013399501
393 0.0010227556542277

678 0.00016285698921205321
679 0.0001620713248538714
680 0.00016129061694519694
681 0.00016051482638075956
682 0.00015974391444162052
683 0.00015897784278506527
684 0.00015821657343413934
685 0.0001574600687680379
686 0.0001567082915125517
687 0.00015596120473191665
688 0.0001552187718210211
689 0.00015448095649896602
690 0.00015374772280385708
691 0.00015301903508844556
692 0.00015229485801738626
693 0.0001515751565649706
694 0.00015085989601451677
695 0.00015014904195782762
696 0.00014944256029592235
697 0.00014874041723973017
698 0.00014804257931134228
699 0.00014734901334496886
700 0.00014665968648792344
701 0.00014597456620120267
702 0.00014529362025943735
703 0.00014461681675014793
704 0.0001439441240726474
705 0.00014327551093545126
706 0.00014261094635350404
707 0.0001419503996443658
708 0.0001412938404234351
709 0.00014064123859900058
710 0.00013999256436648944
711 0.00013934778820205753
712 0.00013870688085681514
713 0.00013806981334999155
714 0.00013743655696276365
715 0.000

In [50]:
print(spam_target.init.numpy())
print(spam_model.init.numpy())
print("--------")
print(spam_target.povm.numpy())
print(spam_model.povm.numpy())

[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]
[[ 1.965e-03+0.000e+00j -7.086e-04-3.434e-02j -5.023e-05-1.687e-05j
   1.188e-03+1.347e-04j]
 [-7.086e-04+3.434e-02j  9.967e-01+0.000e+00j -8.379e-04-6.996e-04j
   1.668e-03+3.468e-02j]
 [-5.023e-05+1.687e-05j -8.379e-04+6.996e-04j  7.517e-06+0.000e+00j
  -3.293e-05-3.832e-05j]
 [ 1.188e-03-1.347e-04j  1.668e-03-3.468e-02j -3.293e-05+3.832e-05j
   1.284e-03+0.000e+00j]]
--------
[[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]

 [[0.+0.j 0.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]

 [[0.+0.j 0.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j 1.+0.j 0.+0.j]
  [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]

 [[0.+0.j 0.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
  [0.+0

In [51]:
np.random.seed(43)
random.seed(43)
tf.random.set_seed(43)

spam_model = SPAM(d=d,
                  optimizer = tf.optimizers.Adam(learning_rate=0.01))

spam_model.pretrain(targets = [init_ideal(d), povm_ideal(d)],
                    num_iter = 1000
                    )

spam_model.train(inputs = inputs_spam,
                 targets = targets_spam,
                 num_iter = 1000,
                 N = N_spam
                 )

  0%|          | 0/1000 [00:00<?, ?it/s]

0 0.13240706569752608
1 0.13094371514205638
2 0.12947551043169653
3 0.12800371149473705
4 0.12652932384203308
5 0.12505280005993874
6 0.1235755702909268
7 0.12209854156678823
8 0.1206182521893936
9 0.11913509962692881
10 0.1176514808329339
11 0.11616821877846187
12 0.11468646104241977
13 0.11319914301274923
14 0.11171208739596075
15 0.11022642278824996
16 0.10873946988810457
17 0.10724941126598153
18 0.10574937288757849
19 0.10424623626069371
20 0.10274214442869763
21 0.10123636989728715
22 0.09972774946161114
23 0.09822236438160553
24 0.09671105661085705
25 0.09519479614188284
26 0.09367691201437281
27 0.09215593518073005
28 0.09063537637485022
29 0.08910949228624483
30 0.08758239177013602
31 0.08605683641766235
32 0.08452488270210609
33 0.08299528459721894
34 0.0814689935183801
35 0.07994614914938682
36 0.07842110773473508
37 0.0769004321089722
38 0.07538640580408904
39 0.07387950052034836
40 0.07238019612587943
41 0.070884864075853
42 0.06939346659166457
43 0.06791096119729115
44 0.

333 0.0004339507579981755
334 0.00043071592403751005
335 0.0004275116251381334
336 0.0004243363335351149
337 0.00042118880694118
338 0.0004178981763700888
339 0.0004146561241498781
340 0.00041146242833179405
341 0.0004083166830867579
342 0.00040521828440428576
343 0.0004021664193950592
344 0.0003991600554181021
345 0.00039619792865877774
346 0.00039327853489776307
347 0.00039040012727911385
348 0.000387560726421173
349 0.00038475814711505356
350 0.00038199004340620324
351 0.0003792539706507383
352 0.00037654745992812735
353 0.00037386809771454833
354 0.0003712136025371196
355 0.0003685818907081847
356 0.0003659711250969944
357 0.0003633797438100285
358 0.0003608064689600396
359 0.0003582502986557227
360 0.00035571048729866905
361 0.00035318651985604793
362 0.0003505620287559304
363 0.0003479753437327771
364 0.0003454269353084364
365 0.0003429169536246802
366 0.0003404452441361946
367 0.00033801136803090294
368 0.0003356146224997935
369 0.0003332540583376853
370 0.00033092849498209294
3

648 7.258928535339034e-05
649 7.226068880887896e-05
650 7.193560284770423e-05
651 7.161402954197678e-05
652 7.129593066440035e-05
653 7.098122918340177e-05
654 7.066981234908344e-05
655 7.03615365451642e-05
656 7.005623363940362e-05
657 6.975371806759495e-05
658 6.945379357057873e-05
659 6.915625853412105e-05
660 6.886090928988018e-05
661 6.85675414054079e-05
662 6.826181891198853e-05
663 6.795981321106682e-05
664 6.766148334150079e-05
665 6.736671729178431e-05
666 6.707534001404539e-05
667 6.678712627516576e-05
668 6.650181626595831e-05
669 6.621913164919159e-05
670 6.593878998532647e-05
671 6.566051616462415e-05
672 6.53840503871377e-05
673 6.510915308713238e-05
674 6.483560774349728e-05
675 6.45502927731821e-05
676 6.426804664930137e-05
677 6.398893403845084e-05
678 6.371297086633359e-05
679 6.344012590751638e-05
680 6.317032326209298e-05
681 6.290344483453663e-05
682 6.263933269299622e-05
683 6.237779190589317e-05
684 6.211859486175979e-05
685 6.186148801804421e-05
686 6.1606201500

976 2.1376930472438106e-05
977 2.13107469363824e-05
978 2.124095383050659e-05
979 2.1171632842154275e-05
980 2.1102931372740212e-05
981 2.103493904673348e-05
982 2.0967659582654344e-05
983 2.0901025314915178e-05
984 2.0834946521740574e-05
985 2.076937014915658e-05
986 2.0704316833031708e-05
987 2.0639874571091847e-05
988 2.0573034992148266e-05
989 2.0507411351196915e-05
990 2.0443000077284025e-05
991 2.037963416647511e-05
992 2.031702054434574e-05
993 2.025482382883878e-05
994 2.019276329020156e-05
995 2.013068304274507e-05
996 2.0068566964312628e-05
997 2.0003931587566493e-05
998 1.9939845377968305e-05
999 1.9876415775578985e-05


  0%|          | 0/1000 [00:00<?, ?it/s]

0 0.00036956106924067783
1 0.0003971330093096518
2 0.00032185912952059235
3 0.0002891498476627403
4 0.00026619853361562536
5 0.00024512685731968304
6 0.00021081976114166912
7 0.00019496146749846171
8 0.00019251738896113664
9 0.00016835634761815557
10 0.00014937377649757295
11 0.00015074324069569585
12 0.00014471188281423489
13 0.00012843975426977134
14 0.00012660009277280716
15 0.0001118907975822205
16 9.494890429826215e-05
17 9.83448171028816e-05
18 8.64954219118614e-05
19 6.844406816696832e-05
20 6.329902635429846e-05
21 5.934402801556718e-05
22 5.419940302208528e-05
23 4.9419429256619815e-05
24 4.464334342960919e-05
25 4.137805550888402e-05
26 4.1342135156624714e-05
27 4.19212130810256e-05
28 3.893147723561355e-05
29 3.650137092608769e-05
30 3.695783158340043e-05
31 3.5550942868595906e-05
32 3.4245666226225055e-05
33 3.2925022307541096e-05
34 3.3585065425446024e-05
35 3.2317683668977166e-05
36 2.9782771656356852e-05
37 2.9911605501117835e-05
38 3.282105870016241e-05
39 3.13746793245

319 0.0002622615847966884
320 0.0002280785964198208
321 9.597130545714144e-05
322 0.00011012201520193344
323 0.00024209968232085155
324 0.0002651405416620698
325 0.00014797190838786
326 9.168887068602326e-05
327 0.00019510570764261591
328 0.00029534004087470136
329 0.0002234895129240968
330 0.00013243597309097656
331 0.00018909296479658697
332 0.0003051765437532414
333 0.0003042128535601451
334 0.00024553563278602303
335 0.00024076680763493775
336 0.0003363026401930064
337 0.00040375668780409787
338 0.00043343769305118323
339 0.00040113043690974767
340 0.00044577947192479096
341 0.0005621973765203337
342 0.0007070934438086074
343 0.0007562925492528783
344 0.0006985833386559484
345 0.0004928827664874561
346 0.0007253034198269619
347 0.001115403625508435
348 0.0010417110410557478
349 0.0006813314530403702
350 0.00035848629925597046
351 0.0005056703626047921
352 0.0007616172214783691
353 0.0008449154529308363
354 0.000674687578815302
355 0.00039252108262821417
356 0.0002959739928959928
35

635 0.00014202304660734184
636 0.00015009902808540935
637 0.00015732919551772766
638 0.00016122444055317146
639 0.00016030830984412154
640 0.0001545234769738497
641 0.0001451970204004216
642 0.00013460406532464958
643 0.00012528898856137115
644 0.00011935550131288907
645 0.00011792090302199984
646 0.00012086370341850736
647 0.0001269070678428648
648 0.0001339986312308935
649 0.00013987999760432834
650 0.0001426929485616075
651 0.0001414613282750478
652 0.00013632804600547028
653 0.00012849733383477337
654 0.00011990439209489079
655 0.00011269399556377977
656 0.00010863392211105907
657 0.00010861270787965203
658 0.00011236468451103651
659 0.0001185213865087795
660 0.00012500493618341146
661 0.0001296645456840173
662 0.000130950775243806
663 0.0001283895197909098
664 0.00012269319859791066
665 0.00011548994796136775
666 0.00010878662681250739
667 0.00010435564369767712
668 0.0001032353685600416
669 0.00010547475234133039
670 0.00011016810976800506
671 0.00011574886444619471
672 0.0001204

952 0.00025915748718961367
953 0.00026213501726744
954 0.0002615339465425355
955 0.0002574667495509274
956 0.00025131730426092747
957 0.0002447280179524299
958 0.0002389554877884356
959 0.0002347498983394963
960 0.00023247640390964766
961 0.00023225267692227755
962 0.0002340149655382056
963 0.00023749197410372985
964 0.0002420831727586641
965 0.0002466907480224892
966 0.0002497484818064008
967 0.0002498568731496724
968 0.0002467448979513022
969 0.00024138125944657597
970 0.00023520453851638947
971 0.0002294503387037848
972 0.00022492503095715557
973 0.00022205785402114222
974 0.0002210194587409179
975 0.0002217994178369667
976 0.00022421097865800023
977 0.00022781728380605928
978 0.00023180290456713295
979 0.00023492701297420408
980 0.00023586358520960726
981 0.00023397258304967044
982 0.00022971896135768043
983 0.00022424368096110516
984 0.0002187171286035686
985 0.00021400175986362454
986 0.0002106103316414272
987 0.0002087867848535136
988 0.00020858673916081513
989 0.000209908101024

In [52]:
print(spam_target.init.numpy())
print(spam_model.init.numpy())
print("--------")
print(spam_target.povm.numpy())
print(spam_model.povm.numpy())

[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]
[[ 9.999e-01+0.000e+00j  2.555e-03-1.814e-03j -1.876e-03+7.556e-04j
  -6.714e-05-3.422e-05j]
 [ 2.555e-03+1.814e-03j  4.255e-05+0.000e+00j -1.665e-05-2.375e-06j
  -4.040e-07-9.287e-07j]
 [-1.876e-03-7.556e-04j -1.665e-05+2.375e-06j  8.269e-06+0.000e+00j
  -3.614e-08+3.127e-07j]
 [-6.714e-05+3.422e-05j -4.040e-07+9.287e-07j -3.614e-08-3.127e-07j
   1.262e-07+0.000e+00j]]
--------
[[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]

 [[0.+0.j 0.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]

 [[0.+0.j 0.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j 1.+0.j 0.+0.j]
  [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]

 [[0.+0.j 0.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
  [0.+0

## Recover SPAM and Map

### Generate True Model, Full POVM and Inital

In [18]:
np.random.seed(44)
random.seed(44)
tf.random.set_seed(44)

n = 3
d = 2**n
rank = d
c1 = 0.9
c2 = 0.9

#prep error and full POVM error
spam_target = SPAM(d=d)

init_target = c1*init_ideal(d) + (1-c1)*spam_target.init
povm_target = c2*povm_ideal(d) + (1-c2)*spam_target.povm

spam_target = SPAM(d=d,
                  init = init_target,
                  povm = povm_target)

kraus_target = KrausMap(
                        U=generate_unitary(d=d), 
                        c=0.5, 
                        d=d, 
                        rank=rank-1,
                        spam = spam_target
                        )

### Generate Synthetic Data without Noise

In [19]:
inputs_spam, _ = generate_pauliInput_circuits(n)

U_prep = inputs_spam
N_spam = U_prep.shape[0]

state = tf.repeat(spam_target.init[None,:,:], N_spam, axis=0)
state = apply_unitary(state, U_prep)
targets_spam = measurement(state, povm = spam_target.povm)

inputs_map, _ = generate_pauli_circuits(n = n, 
                                    circuit_target=None, 
                                    N = None, 
                                    trace=False)
U_prep, U_basis = inputs_map

N_map = U_prep.shape[0]
state = tf.repeat(tf.expand_dims(spam_target.init, axis=0), N_map, axis=0)
state = apply_unitary(state, U_prep)
state = kraus_target.apply_map(state)
targets_map = measurement(state, U_basis, spam_target.povm)

### Fit Model, No SPAM

In [20]:
np.random.seed(45)
random.seed(45)
tf.random.set_seed(45)

kraus_model = KrausMap(
                       d = d, 
                       rank = rank,
                       spam = None,
                       )

model = ModelQuantumMap(
                        q_map = kraus_model,
                        loss = probs_loss,
                        optimizer = tf.optimizers.Adam(learning_rate=0.01),
                        )

model.train(inputs = inputs_map,
            targets = targets_map,
            #inputs_val = inputs_map,
            #targets_val = targets_map,
            num_iter = 1000,
            N = 1000
            )

  0%|          | 0/1000 [00:00<?, ?it/s]

Step:0, train: 0.26495, val: 0.00000
Step:1, train: 0.25523, val: 0.00000
Step:2, train: 0.25237, val: 0.00000
Step:3, train: 0.26144, val: 0.00000
Step:4, train: 0.24677, val: 0.00000
Step:5, train: 0.24449, val: 0.00000
Step:6, train: 0.24607, val: 0.00000
Step:7, train: 0.23934, val: 0.00000
Step:8, train: 0.23059, val: 0.00000
Step:9, train: 0.22965, val: 0.00000
Step:10, train: 0.22543, val: 0.00000
Step:11, train: 0.22433, val: 0.00000
Step:12, train: 0.22278, val: 0.00000
Step:13, train: 0.22132, val: 0.00000
Step:14, train: 0.21625, val: 0.00000
Step:15, train: 0.21054, val: 0.00000
Step:16, train: 0.20771, val: 0.00000
Step:17, train: 0.20693, val: 0.00000
Step:18, train: 0.20037, val: 0.00000
Step:19, train: 0.19678, val: 0.00000
Step:20, train: 0.20114, val: 0.00000
Step:21, train: 0.19527, val: 0.00000
Step:22, train: 0.19613, val: 0.00000
Step:23, train: 0.18909, val: 0.00000
Step:24, train: 0.18512, val: 0.00000
Step:25, train: 0.18408, val: 0.00000
Step:26, train: 0.1811

Step:216, train: 0.02015, val: 0.00000
Step:217, train: 0.01961, val: 0.00000
Step:218, train: 0.01960, val: 0.00000
Step:219, train: 0.01933, val: 0.00000
Step:220, train: 0.01977, val: 0.00000
Step:221, train: 0.01983, val: 0.00000
Step:222, train: 0.01958, val: 0.00000
Step:223, train: 0.01871, val: 0.00000
Step:224, train: 0.01884, val: 0.00000
Step:225, train: 0.01927, val: 0.00000
Step:226, train: 0.01988, val: 0.00000
Step:227, train: 0.01819, val: 0.00000
Step:228, train: 0.01881, val: 0.00000
Step:229, train: 0.01867, val: 0.00000
Step:230, train: 0.01884, val: 0.00000
Step:231, train: 0.01847, val: 0.00000
Step:232, train: 0.01806, val: 0.00000
Step:233, train: 0.01842, val: 0.00000
Step:234, train: 0.01807, val: 0.00000
Step:235, train: 0.01814, val: 0.00000
Step:236, train: 0.01804, val: 0.00000
Step:237, train: 0.01797, val: 0.00000
Step:238, train: 0.01800, val: 0.00000
Step:239, train: 0.01778, val: 0.00000
Step:240, train: 0.01746, val: 0.00000
Step:241, train: 0.01854,

Step:431, train: 0.00884, val: 0.00000
Step:432, train: 0.00883, val: 0.00000
Step:433, train: 0.00868, val: 0.00000
Step:434, train: 0.00873, val: 0.00000
Step:435, train: 0.00847, val: 0.00000
Step:436, train: 0.00836, val: 0.00000
Step:437, train: 0.00832, val: 0.00000
Step:438, train: 0.00877, val: 0.00000
Step:439, train: 0.00858, val: 0.00000
Step:440, train: 0.00822, val: 0.00000
Step:441, train: 0.00838, val: 0.00000
Step:442, train: 0.00867, val: 0.00000
Step:443, train: 0.00869, val: 0.00000
Step:444, train: 0.00836, val: 0.00000
Step:445, train: 0.00831, val: 0.00000
Step:446, train: 0.00848, val: 0.00000
Step:447, train: 0.00820, val: 0.00000
Step:448, train: 0.00816, val: 0.00000
Step:449, train: 0.00855, val: 0.00000
Step:450, train: 0.00833, val: 0.00000
Step:451, train: 0.00830, val: 0.00000
Step:452, train: 0.00827, val: 0.00000
Step:453, train: 0.00821, val: 0.00000
Step:454, train: 0.00828, val: 0.00000
Step:455, train: 0.00823, val: 0.00000
Step:456, train: 0.00822,

Step:642, train: 0.00541, val: 0.00000
Step:643, train: 0.00536, val: 0.00000
Step:644, train: 0.00542, val: 0.00000
Step:645, train: 0.00536, val: 0.00000
Step:646, train: 0.00532, val: 0.00000
Step:647, train: 0.00532, val: 0.00000
Step:648, train: 0.00529, val: 0.00000
Step:649, train: 0.00530, val: 0.00000
Step:650, train: 0.00527, val: 0.00000
Step:651, train: 0.00523, val: 0.00000
Step:652, train: 0.00549, val: 0.00000
Step:653, train: 0.00538, val: 0.00000
Step:654, train: 0.00530, val: 0.00000
Step:655, train: 0.00546, val: 0.00000
Step:656, train: 0.00531, val: 0.00000
Step:657, train: 0.00515, val: 0.00000
Step:658, train: 0.00515, val: 0.00000
Step:659, train: 0.00526, val: 0.00000
Step:660, train: 0.00522, val: 0.00000
Step:661, train: 0.00520, val: 0.00000
Step:662, train: 0.00538, val: 0.00000
Step:663, train: 0.00526, val: 0.00000
Step:664, train: 0.00521, val: 0.00000
Step:665, train: 0.00533, val: 0.00000
Step:666, train: 0.00541, val: 0.00000
Step:667, train: 0.00505,

Step:854, train: 0.00430, val: 0.00000
Step:855, train: 0.00419, val: 0.00000
Step:856, train: 0.00420, val: 0.00000
Step:857, train: 0.00421, val: 0.00000
Step:858, train: 0.00421, val: 0.00000
Step:859, train: 0.00420, val: 0.00000
Step:860, train: 0.00413, val: 0.00000
Step:861, train: 0.00420, val: 0.00000
Step:862, train: 0.00424, val: 0.00000
Step:863, train: 0.00411, val: 0.00000
Step:864, train: 0.00400, val: 0.00000
Step:865, train: 0.00418, val: 0.00000
Step:866, train: 0.00430, val: 0.00000
Step:867, train: 0.00422, val: 0.00000
Step:868, train: 0.00439, val: 0.00000
Step:869, train: 0.00407, val: 0.00000
Step:870, train: 0.00417, val: 0.00000
Step:871, train: 0.00411, val: 0.00000
Step:872, train: 0.00414, val: 0.00000
Step:873, train: 0.00418, val: 0.00000
Step:874, train: 0.00430, val: 0.00000
Step:875, train: 0.00414, val: 0.00000
Step:876, train: 0.00423, val: 0.00000
Step:877, train: 0.00427, val: 0.00000
Step:878, train: 0.00412, val: 0.00000
Step:879, train: 0.00412,

In [21]:
channel_fidelity(kraus_model, kraus_target).numpy()

0.9324280060672356

### Fit Model, Full Initial and POVM

In [22]:
np.random.seed(45)
random.seed(45)
tf.random.set_seed(45)

spam_model = SPAM(d=d,
#                  use_corr_mat=True,
                  optimizer = tf.optimizers.Adam(learning_rate=0.01))

spam_model.pretrain(targets = [init_ideal(d), povm_ideal(d)],
                    num_iter = 1000,
                    )

spam_model.train(inputs = inputs_spam,
                 targets = targets_spam,
                 num_iter = 1000,
                 N = N_spam
                )

kraus_model = KrausMap(
                       d = d, 
                       rank = rank,
                       spam = spam_model,
                       )

model = ModelQuantumMap(
                        q_map = kraus_model,
                        loss = probs_loss,
                        optimizer = tf.optimizers.Adam(learning_rate=0.01),
                        )

model.train(inputs = inputs_map,
            targets = targets_map,
            #inputs_val = inputs_map,
            #targets_val = targets_map,
            num_iter = 1000,
            N = 1000
            )

  0%|          | 0/1000 [00:00<?, ?it/s]

0 0.03141456756929366
1 0.031166975276331055
2 0.030916752796686753
3 0.030664585189758266
4 0.030410662516107485
5 0.030154870372947576
6 0.029897309762168256
7 0.02963806614652993
8 0.02937681139125036
9 0.02911354777404548
10 0.028848363984930334
11 0.028581186125615797
12 0.028311642995783124
13 0.02803961692382042
14 0.027764981299482495
15 0.027486983029764912
16 0.027205651959188677
17 0.02692028273727074
18 0.02663117095230913
19 0.02633809586244357
20 0.026041232249029295
21 0.025741038233907665
22 0.025437229678864887
23 0.025129361173036005
24 0.02481592867124459
25 0.024497700400147468
26 0.024174843014383517
27 0.0238475057642159
28 0.02351614036917899
29 0.023180251449212225
30 0.022840539469906385
31 0.022496857064196796
32 0.022148732101594598
33 0.021796851142691046
34 0.02144083333224076
35 0.021081066007830183
36 0.020717367075350848
37 0.020349667184994057
38 0.019978074465031033
39 0.01960379939076555
40 0.01922724990401304
41 0.018848439186111638
42 0.018467638016

328 5.06261728141515e-05
329 5.0171911397581006e-05
330 4.972300294613762e-05
331 4.927926393583407e-05
332 4.884053721903414e-05
333 4.840655136731511e-05
334 4.797741718609095e-05
335 4.7552950180202404e-05
336 4.7133299607357e-05
337 4.6718466904429164e-05
338 4.6308450247410405e-05
339 4.590314310317884e-05
340 4.550264844520289e-05
341 4.510693836503372e-05
342 4.47159603975153e-05
343 4.432966305229882e-05
344 4.3947968252779906e-05
345 4.357067479103437e-05
346 4.31978144790701e-05
347 4.282929252408123e-05
348 4.246487744882403e-05
349 4.210462003444871e-05
350 4.174837606665283e-05
351 4.139614905291012e-05
352 4.104787816298747e-05
353 4.0703502891573544e-05
354 4.03628945831105e-05
355 4.002606736995506e-05
356 3.969297334277973e-05
357 3.9363580932141966e-05
358 3.9037870249342185e-05
359 3.8715746238194134e-05
360 3.8397203033576305e-05
361 3.808218284583175e-05
362 3.777048623127574e-05
363 3.746222509244208e-05
364 3.7157354497459914e-05
365 3.685576307840385e-05
366 3.6

645 6.628758571358038e-06
646 6.597248573958805e-06
647 6.565932741816406e-06
648 6.5348142248708376e-06
649 6.503888053642475e-06
650 6.473146002821147e-06
651 6.442593926211538e-06
652 6.412235543188227e-06
653 6.382043778724522e-06
654 6.35204237015655e-06
655 6.322216400535703e-06
656 6.292570418903115e-06
657 6.263109541935188e-06
658 6.233829838821358e-06
659 6.204721186890207e-06
660 6.1757770698171e-06
661 6.147009011386293e-06
662 6.118415697008465e-06
663 6.089998286436506e-06
664 6.061753362598003e-06
665 6.0336802408565e-06
666 6.00576862472574e-06
667 5.978013177743118e-06
668 5.950424507913867e-06
669 5.923001202004672e-06
670 5.89574514066172e-06
671 5.868653169374909e-06
672 5.841724513072929e-06
673 5.814949112932039e-06
674 5.788322121199868e-06
675 5.761853986866802e-06
676 5.735542906198591e-06
677 5.709392989610261e-06
678 5.683399089334031e-06
679 5.657550667923268e-06
680 5.631847512598969e-06
681 5.606295845132863e-06
682 5.580896458552645e-06
683 5.555647501044

961 1.9016098001545578e-06
962 1.895292272073082e-06
963 1.8890015958922154e-06
964 1.8827387367909853e-06
965 1.8765032313636392e-06
966 1.8702910181285628e-06
967 1.8641056126313046e-06
968 1.8579458343162898e-06
969 1.8518119663507985e-06
970 1.8457034584384082e-06
971 1.839621812218563e-06
972 1.8335668066680892e-06
973 1.8275323980681039e-06
974 1.8215235081299418e-06
975 1.815540439622523e-06
976 1.809582718232842e-06
977 1.8036500099055784e-06
978 1.79774233471663e-06
979 1.7918592983641616e-06
980 1.7860014951213385e-06
981 1.7801682675015927e-06
982 1.7743555766119624e-06
983 1.7685675771169565e-06
984 1.762803150698112e-06
985 1.757062801292919e-06
986 1.7513469113756401e-06
987 1.745654076808506e-06
988 1.7399857485407442e-06
989 1.7343415888341408e-06
990 1.7287183819911109e-06
991 1.7231187358405438e-06
992 1.7175421477820534e-06
993 1.7119869449486982e-06
994 1.7064550781838883e-06
995 1.7009468204882657e-06
996 1.695461837080333e-06
997 1.6899961944377512e-06
998 1.68455

...

Step:0, train: 0.22360, val: 0.00000
Step:1, train: 0.22121, val: 0.00000
Step:2, train: 0.21635, val: 0.00000
Step:3, train: 0.21048, val: 0.00000
Step:4, train: 0.20934, val: 0.00000
Step:5, train: 0.20693, val: 0.00000
Step:6, train: 0.20536, val: 0.00000
Step:7, train: 0.20666, val: 0.00000
Step:8, train: 0.20431, val: 0.00000
Step:9, train: 0.20100, val: 0.00000
Step:10, train: 0.19937, val: 0.00000
Step:11, train: 0.19848, val: 0.00000
Step:12, train: 0.18729, val: 0.00000
Step:13, train: 0.18705, val: 0.00000
Step:14, train: 0.18827, val: 0.00000
Step:15, train: 0.18529, val: 0.00000
Step:16, train: 0.17992, val: 0.00000
Step:17, train: 0.17699, val: 0.00000
Step:18, train: 0.17651, val: 0.00000
Step:19, train: 0.17607, val: 0.00000
Step:20, train: 0.16990, val: 0.00000
Step:21, train: 0.17266, val: 0.00000
Step:22, train: 0.17289, val: 0.00000
Step:23, train: 0.17177, val: 0.00000
Step:24, train: 0.16634, val: 0.00000
Step:25, train: 0.16583, val: 0.00000
Step:26, train: 0.1646

Step:216, train: 0.01104, val: 0.00000
Step:217, train: 0.01069, val: 0.00000
Step:218, train: 0.01092, val: 0.00000
Step:219, train: 0.01082, val: 0.00000
Step:220, train: 0.01061, val: 0.00000
Step:221, train: 0.01072, val: 0.00000
Step:222, train: 0.01063, val: 0.00000
Step:223, train: 0.01040, val: 0.00000
Step:224, train: 0.01069, val: 0.00000
Step:225, train: 0.01063, val: 0.00000
Step:226, train: 0.01042, val: 0.00000
Step:227, train: 0.01010, val: 0.00000
Step:228, train: 0.01004, val: 0.00000
Step:229, train: 0.01013, val: 0.00000
Step:230, train: 0.01015, val: 0.00000
Step:231, train: 0.00997, val: 0.00000
Step:232, train: 0.00990, val: 0.00000
Step:233, train: 0.01008, val: 0.00000
Step:234, train: 0.00999, val: 0.00000
Step:235, train: 0.00961, val: 0.00000
Step:236, train: 0.00941, val: 0.00000
Step:237, train: 0.00948, val: 0.00000
Step:238, train: 0.00935, val: 0.00000
Step:239, train: 0.00913, val: 0.00000
Step:240, train: 0.00932, val: 0.00000
Step:241, train: 0.00921,

Step:432, train: 0.00332, val: 0.00000
Step:433, train: 0.00332, val: 0.00000
Step:434, train: 0.00318, val: 0.00000
Step:435, train: 0.00327, val: 0.00000
Step:436, train: 0.00328, val: 0.00000
Step:437, train: 0.00315, val: 0.00000
Step:438, train: 0.00313, val: 0.00000
Step:439, train: 0.00316, val: 0.00000
Step:440, train: 0.00324, val: 0.00000
Step:441, train: 0.00321, val: 0.00000
Step:442, train: 0.00319, val: 0.00000
Step:443, train: 0.00312, val: 0.00000
Step:444, train: 0.00309, val: 0.00000
Step:445, train: 0.00307, val: 0.00000
Step:446, train: 0.00316, val: 0.00000
Step:447, train: 0.00298, val: 0.00000
Step:448, train: 0.00305, val: 0.00000
Step:449, train: 0.00296, val: 0.00000
Step:450, train: 0.00297, val: 0.00000
Step:451, train: 0.00298, val: 0.00000
Step:452, train: 0.00299, val: 0.00000
Step:453, train: 0.00295, val: 0.00000
Step:454, train: 0.00283, val: 0.00000
Step:455, train: 0.00294, val: 0.00000
Step:456, train: 0.00291, val: 0.00000
Step:457, train: 0.00288,

Step:647, train: 0.00088, val: 0.00000
Step:648, train: 0.00088, val: 0.00000
Step:649, train: 0.00085, val: 0.00000
Step:650, train: 0.00086, val: 0.00000
Step:651, train: 0.00084, val: 0.00000
Step:652, train: 0.00087, val: 0.00000
Step:653, train: 0.00082, val: 0.00000
Step:654, train: 0.00085, val: 0.00000
Step:655, train: 0.00083, val: 0.00000
Step:656, train: 0.00083, val: 0.00000
Step:657, train: 0.00080, val: 0.00000
Step:658, train: 0.00085, val: 0.00000
Step:659, train: 0.00082, val: 0.00000
Step:660, train: 0.00082, val: 0.00000
Step:661, train: 0.00080, val: 0.00000
Step:662, train: 0.00080, val: 0.00000
Step:663, train: 0.00080, val: 0.00000
Step:664, train: 0.00078, val: 0.00000
Step:665, train: 0.00079, val: 0.00000
Step:666, train: 0.00078, val: 0.00000
Step:667, train: 0.00078, val: 0.00000
Step:668, train: 0.00077, val: 0.00000
Step:669, train: 0.00076, val: 0.00000
Step:670, train: 0.00077, val: 0.00000
Step:671, train: 0.00077, val: 0.00000
Step:672, train: 0.00075,

Step:858, train: 0.00025, val: 0.00000
Step:859, train: 0.00025, val: 0.00000
Step:860, train: 0.00025, val: 0.00000
Step:861, train: 0.00024, val: 0.00000
Step:862, train: 0.00025, val: 0.00000
Step:863, train: 0.00024, val: 0.00000
Step:864, train: 0.00024, val: 0.00000
Step:865, train: 0.00024, val: 0.00000
Step:866, train: 0.00023, val: 0.00000
Step:867, train: 0.00024, val: 0.00000
Step:868, train: 0.00024, val: 0.00000
Step:869, train: 0.00023, val: 0.00000
Step:870, train: 0.00023, val: 0.00000
Step:871, train: 0.00023, val: 0.00000
Step:872, train: 0.00023, val: 0.00000
Step:873, train: 0.00022, val: 0.00000
Step:874, train: 0.00023, val: 0.00000
Step:875, train: 0.00023, val: 0.00000
Step:876, train: 0.00023, val: 0.00000
Step:877, train: 0.00022, val: 0.00000
Step:878, train: 0.00022, val: 0.00000
Step:879, train: 0.00022, val: 0.00000
Step:880, train: 0.00022, val: 0.00000
Step:881, train: 0.00021, val: 0.00000
Step:882, train: 0.00022, val: 0.00000
Step:883, train: 0.00022,

In [23]:
channel_fidelity(kraus_model, kraus_target).numpy()

0.9974435073023422

### Fit Model, Initial and Corruption Matrix

In [24]:
np.random.seed(45)
random.seed(45)
tf.random.set_seed(45)

spam_model = SPAM(d=d,
                  use_corr_mat=True,
                  optimizer = tf.optimizers.Adam(learning_rate=0.01))

spam_model.pretrain(targets = [init_ideal(d), povm_ideal(d)],
                    num_iter = 1000,
                    )

spam_model.train(inputs = inputs_spam,
                 targets = targets_spam,
                 num_iter = 1000,
                 N = N_spam
                )




kraus_model = KrausMap(
                       d = d, 
                       rank = rank,
                       spam = spam_model,
                       )

model = ModelQuantumMap(
                        q_map = kraus_model,
                        loss = probs_loss,
                        optimizer = tf.optimizers.Adam(learning_rate=0.01),
                        )

model.train(inputs = inputs_map,
            targets = targets_map,
            #inputs_val = inputs_map,
            #targets_val = targets_map,
            num_iter = 1000,
            N = 1000
            )

  0%|          | 0/1000 [00:00<?, ?it/s]

0 0.030847143338659015
1 0.030662459515911837
2 0.030478113635720658
3 0.030294378560601604
4 0.03011125446539551
5 0.0299286394951683
6 0.02974637630255854
7 0.029564343058466922
8 0.029382166271182886
9 0.029199761531843116
10 0.029017001016869184
11 0.028834124534501303
12 0.028650785940328858
13 0.028466479769947366
14 0.028281007779332172
15 0.02809361896369352
16 0.027905004486438477
17 0.027714040522829386
18 0.027520756143509136
19 0.027325171289391166
20 0.02712710205235179
21 0.02692724941940549
22 0.02672547779844971
23 0.026520626423110247
24 0.026311699381119276
25 0.026099274888549243
26 0.025883769335170993
27 0.025665682273699086
28 0.02544524040096212
29 0.025222609140702103
30 0.024996647170454134
31 0.024767377827047465
32 0.02453436545109277
33 0.024298003404903394
34 0.024058226701104274
35 0.023815564230044564
36 0.02356915406288541
37 0.023319200323779987
38 0.02306544890680265
39 0.022809350387594865
40 0.022550724877813944
41 0.022290167802329452
42 0.022026576

331 3.6511596261629186e-05
332 3.618657138621934e-05
333 3.5865092004874195e-05
334 3.554710002206558e-05
335 3.523259201050358e-05
336 3.492157428992078e-05
337 3.461406243846226e-05
338 3.431007868772293e-05
339 3.40096365743444e-05
340 3.371275055427515e-05
341 3.341941368945111e-05
342 3.312958760193579e-05
343 3.284322786155081e-05
344 3.256027993065162e-05
345 3.2280679313774234e-05
346 3.2004357287621025e-05
347 3.173124784660386e-05
348 3.146128804622312e-05
349 3.119440988846168e-05
350 3.093055106659208e-05
351 3.06696514514125e-05
352 3.0411660445555467e-05
353 3.0156524752494703e-05
354 2.9904201138563216e-05
355 2.9654650381537954e-05
356 2.94078432582295e-05
357 2.916375178709954e-05
358 2.8922351681273416e-05
359 2.8683619722250747e-05
360 2.844752731719148e-05
361 2.8214044712288655e-05
362 2.798314104155141e-05
363 2.775477967594176e-05
364 2.7528921813867278e-05
365 2.7305525918907145e-05
366 2.708454976554745e-05
367 2.6865951416733643e-05
368 2.664968946662165e-05
3

643 4.952536888614449e-06
644 4.928943245406903e-06
645 4.9054940436880885e-06
646 4.8821881326770954e-06
647 4.859024380256441e-06
648 4.836001674915553e-06
649 4.813118927080195e-06
650 4.790375069785903e-06
651 4.7677690586835265e-06
652 4.745299871408363e-06
653 4.722966506364772e-06
654 4.7007679810144976e-06
655 4.678703329774621e-06
656 4.6567716016469145e-06
657 4.634971857709996e-06
658 4.61330316860637e-06
659 4.591764612150547e-06
660 4.570355271171402e-06
661 4.54907423168667e-06
662 4.527920581481385e-06
663 4.506893409139213e-06
664 4.4859918035468605e-06
665 4.465214853866859e-06
666 4.444561649942291e-06
667 4.424031283084067e-06
668 4.403622847159602e-06
669 4.38333543990124e-06
670 4.363168164333473e-06
671 4.343120130222465e-06
672 4.3231904554524225e-06
673 4.3033782672445005e-06
674 4.283682703144606e-06
675 4.26410291172888e-06
676 4.244638052991568e-06
677 4.2252872984058985e-06
678 4.2060498306661835e-06
679 4.186924843142382e-06
680 4.1679115390936005e-06
681 4

963 1.4024420753011894e-06
964 1.3978124133551872e-06
965 1.3932020907772523e-06
966 1.3886110009957175e-06
967 1.38403903859182e-06
968 1.3794860996504376e-06
969 1.3749520820619338e-06
970 1.370436885752916e-06
971 1.3659404128347675e-06
972 1.3614625676577853e-06
973 1.3570032567705427e-06
974 1.352562388784008e-06
975 1.348139874152169e-06
976 1.3437356248805604e-06
977 1.3393495541853374e-06
978 1.3349815761217576e-06
979 1.3306316052082805e-06
980 1.3262995560699416e-06
981 1.321985343124364e-06
982 1.317688880328323e-06
983 1.3134100810013127e-06
984 1.3091488577342158e-06
985 1.3049051223869263e-06
986 1.3006787861713385e-06
987 1.2964697598092613e-06
988 1.2922779537532967e-06
989 1.288103278449292e-06
990 1.2839456446217356e-06
991 1.279804963558158e-06
992 1.2756811473731555e-06
993 1.2715741092316623e-06
994 1.2674837635177417e-06
995 1.263410025937082e-06
996 1.2593528135501987e-06
997 1.255312044735514e-06
998 1.2512876390894938e-06
999 1.2472795172747822e-06


...

Step:0, train: 0.21669, val: 0.00000
Step:1, train: 0.21973, val: 0.00000
Step:2, train: 0.21995, val: 0.00000
Step:3, train: 0.21192, val: 0.00000
Step:4, train: 0.20785, val: 0.00000
Step:5, train: 0.20720, val: 0.00000
Step:6, train: 0.20829, val: 0.00000
Step:7, train: 0.20108, val: 0.00000
Step:8, train: 0.20012, val: 0.00000
Step:9, train: 0.19640, val: 0.00000
Step:10, train: 0.19646, val: 0.00000
Step:11, train: 0.19878, val: 0.00000
Step:12, train: 0.18694, val: 0.00000
Step:13, train: 0.18159, val: 0.00000
Step:14, train: 0.18705, val: 0.00000
Step:15, train: 0.18530, val: 0.00000
Step:16, train: 0.18412, val: 0.00000
Step:17, train: 0.17866, val: 0.00000
Step:18, train: 0.17230, val: 0.00000
Step:19, train: 0.17398, val: 0.00000
Step:20, train: 0.17152, val: 0.00000
Step:21, train: 0.17282, val: 0.00000
Step:22, train: 0.16870, val: 0.00000
Step:23, train: 0.17115, val: 0.00000
Step:24, train: 0.16891, val: 0.00000
Step:25, train: 0.16212, val: 0.00000
Step:26, train: 0.1682

Step:217, train: 0.01194, val: 0.00000
Step:218, train: 0.01150, val: 0.00000
Step:219, train: 0.01168, val: 0.00000
Step:220, train: 0.01165, val: 0.00000
Step:221, train: 0.01124, val: 0.00000
Step:222, train: 0.01141, val: 0.00000
Step:223, train: 0.01150, val: 0.00000
Step:224, train: 0.01079, val: 0.00000
Step:225, train: 0.01135, val: 0.00000
Step:226, train: 0.01085, val: 0.00000
Step:227, train: 0.01116, val: 0.00000
Step:228, train: 0.01039, val: 0.00000
Step:229, train: 0.01095, val: 0.00000
Step:230, train: 0.01078, val: 0.00000
Step:231, train: 0.01060, val: 0.00000
Step:232, train: 0.01009, val: 0.00000
Step:233, train: 0.01097, val: 0.00000
Step:234, train: 0.01037, val: 0.00000
Step:235, train: 0.01045, val: 0.00000
Step:236, train: 0.01061, val: 0.00000
Step:237, train: 0.01025, val: 0.00000
Step:238, train: 0.01013, val: 0.00000
Step:239, train: 0.01005, val: 0.00000
Step:240, train: 0.01025, val: 0.00000
Step:241, train: 0.01000, val: 0.00000
Step:242, train: 0.01038,

Step:433, train: 0.00306, val: 0.00000
Step:434, train: 0.00303, val: 0.00000
Step:435, train: 0.00312, val: 0.00000
Step:436, train: 0.00291, val: 0.00000
Step:437, train: 0.00288, val: 0.00000
Step:438, train: 0.00294, val: 0.00000
Step:439, train: 0.00286, val: 0.00000
Step:440, train: 0.00286, val: 0.00000
Step:441, train: 0.00286, val: 0.00000
Step:442, train: 0.00299, val: 0.00000
Step:443, train: 0.00275, val: 0.00000
Step:444, train: 0.00288, val: 0.00000
Step:445, train: 0.00284, val: 0.00000
Step:446, train: 0.00283, val: 0.00000
Step:447, train: 0.00279, val: 0.00000
Step:448, train: 0.00274, val: 0.00000
Step:449, train: 0.00278, val: 0.00000
Step:450, train: 0.00281, val: 0.00000
Step:451, train: 0.00266, val: 0.00000
Step:452, train: 0.00270, val: 0.00000
Step:453, train: 0.00270, val: 0.00000
Step:454, train: 0.00264, val: 0.00000
Step:455, train: 0.00262, val: 0.00000
Step:456, train: 0.00269, val: 0.00000
Step:457, train: 0.00268, val: 0.00000
Step:458, train: 0.00247,

Step:646, train: 0.00103, val: 0.00000
Step:647, train: 0.00102, val: 0.00000
Step:648, train: 0.00104, val: 0.00000
Step:649, train: 0.00105, val: 0.00000
Step:650, train: 0.00099, val: 0.00000
Step:651, train: 0.00102, val: 0.00000
Step:652, train: 0.00102, val: 0.00000
Step:653, train: 0.00099, val: 0.00000
Step:654, train: 0.00102, val: 0.00000
Step:655, train: 0.00103, val: 0.00000
Step:656, train: 0.00097, val: 0.00000
Step:657, train: 0.00095, val: 0.00000
Step:658, train: 0.00100, val: 0.00000
Step:659, train: 0.00096, val: 0.00000
Step:660, train: 0.00097, val: 0.00000
Step:661, train: 0.00097, val: 0.00000
Step:662, train: 0.00094, val: 0.00000
Step:663, train: 0.00096, val: 0.00000
Step:664, train: 0.00092, val: 0.00000
Step:665, train: 0.00093, val: 0.00000
Step:666, train: 0.00094, val: 0.00000
Step:667, train: 0.00091, val: 0.00000
Step:668, train: 0.00094, val: 0.00000
Step:669, train: 0.00091, val: 0.00000
Step:670, train: 0.00092, val: 0.00000
Step:671, train: 0.00090,

Step:860, train: 0.00038, val: 0.00000
Step:861, train: 0.00037, val: 0.00000
Step:862, train: 0.00037, val: 0.00000
Step:863, train: 0.00036, val: 0.00000
Step:864, train: 0.00037, val: 0.00000
Step:865, train: 0.00037, val: 0.00000
Step:866, train: 0.00037, val: 0.00000
Step:867, train: 0.00037, val: 0.00000
Step:868, train: 0.00036, val: 0.00000
Step:869, train: 0.00037, val: 0.00000
Step:870, train: 0.00037, val: 0.00000
Step:871, train: 0.00035, val: 0.00000
Step:872, train: 0.00036, val: 0.00000
Step:873, train: 0.00036, val: 0.00000
Step:874, train: 0.00036, val: 0.00000
Step:875, train: 0.00035, val: 0.00000
Step:876, train: 0.00036, val: 0.00000
Step:877, train: 0.00035, val: 0.00000
Step:878, train: 0.00036, val: 0.00000
Step:879, train: 0.00036, val: 0.00000
Step:880, train: 0.00034, val: 0.00000
Step:881, train: 0.00035, val: 0.00000
Step:882, train: 0.00035, val: 0.00000
Step:883, train: 0.00034, val: 0.00000
Step:884, train: 0.00035, val: 0.00000
Step:885, train: 0.00035,

In [25]:
channel_fidelity(kraus_model, kraus_target).numpy()

0.9971380111753588

### Generate Synthetic Data with Noise

In [26]:
np.random.seed(44)
random.seed(44)
tf.random.set_seed(44)

inputs_spam, _ = generate_pauliInput_circuits(n)

U_prep = inputs_spam
N_spam = U_prep.shape[0]

state = tf.repeat(spam_target.init[None,:,:], N_spam, axis=0)
state = apply_unitary(state, U_prep)
targets_spam = measurement(state, povm = spam_target.povm)

#add noise
targets_spam = targets_spam + tf.cast(tf.random.normal(targets_spam.shape, 0, 0.02), dtype = precision)

inputs_map, _ = generate_pauli_circuits(n = n, 
                                    circuit_target=None, 
                                    N = None, 
                                    trace=False)
U_prep, U_basis = inputs_map

N_map = U_prep.shape[0]
state = tf.repeat(tf.expand_dims(spam_target.init, axis=0), N_map, axis=0)
state = apply_unitary(state, U_prep)
state = kraus_target.apply_map(state)
targets_map = measurement(state, U_basis, spam_target.povm)

#add noise
targets_map = targets_map + tf.cast(tf.random.normal(targets_map.shape, 0, 0.02), dtype = precision)

### Fit Model, No SPAM

In [27]:
np.random.seed(45)
random.seed(45)
tf.random.set_seed(45)

kraus_model = KrausMap(
                       d = d, 
                       rank = rank,
                       spam = None,
                       )

model = ModelQuantumMap(
                        q_map = kraus_model,
                        loss = probs_loss,
                        optimizer = tf.optimizers.Adam(learning_rate=0.01),
                        )

model.train(inputs = inputs_map,
            targets = targets_map,
            #inputs_val = inputs_map,
            #targets_val = targets_map,
            num_iter = 1000,
            N = 1000
            )

  0%|          | 0/1000 [00:00<?, ?it/s]

Step:0, train: 0.29031, val: 0.00000
Step:1, train: 0.27971, val: 0.00000
Step:2, train: 0.27838, val: 0.00000
Step:3, train: 0.28383, val: 0.00000
Step:4, train: 0.26939, val: 0.00000
Step:5, train: 0.26980, val: 0.00000
Step:6, train: 0.26939, val: 0.00000
Step:7, train: 0.26708, val: 0.00000
Step:8, train: 0.25550, val: 0.00000
Step:9, train: 0.25431, val: 0.00000
Step:10, train: 0.24699, val: 0.00000
Step:11, train: 0.25017, val: 0.00000
Step:12, train: 0.24472, val: 0.00000
Step:13, train: 0.24425, val: 0.00000
Step:14, train: 0.24199, val: 0.00000
Step:15, train: 0.23607, val: 0.00000
Step:16, train: 0.23106, val: 0.00000
Step:17, train: 0.22956, val: 0.00000
Step:18, train: 0.22557, val: 0.00000
Step:19, train: 0.22164, val: 0.00000
Step:20, train: 0.22635, val: 0.00000
Step:21, train: 0.21955, val: 0.00000
Step:22, train: 0.22302, val: 0.00000
Step:23, train: 0.21432, val: 0.00000
Step:24, train: 0.21059, val: 0.00000
Step:25, train: 0.20888, val: 0.00000
Step:26, train: 0.2053

Step:216, train: 0.04556, val: 0.00000
Step:217, train: 0.04387, val: 0.00000
Step:218, train: 0.04479, val: 0.00000
Step:219, train: 0.04433, val: 0.00000
Step:220, train: 0.04544, val: 0.00000
Step:221, train: 0.04462, val: 0.00000
Step:222, train: 0.04594, val: 0.00000
Step:223, train: 0.04367, val: 0.00000
Step:224, train: 0.04398, val: 0.00000
Step:225, train: 0.04335, val: 0.00000
Step:226, train: 0.04486, val: 0.00000
Step:227, train: 0.04377, val: 0.00000
Step:228, train: 0.04433, val: 0.00000
Step:229, train: 0.04457, val: 0.00000
Step:230, train: 0.04403, val: 0.00000
Step:231, train: 0.04357, val: 0.00000
Step:232, train: 0.04223, val: 0.00000
Step:233, train: 0.04325, val: 0.00000
Step:234, train: 0.04282, val: 0.00000
Step:235, train: 0.04312, val: 0.00000
Step:236, train: 0.04366, val: 0.00000
Step:237, train: 0.04385, val: 0.00000
Step:238, train: 0.04274, val: 0.00000
Step:239, train: 0.04264, val: 0.00000
Step:240, train: 0.04276, val: 0.00000
Step:241, train: 0.04292,

Step:428, train: 0.03295, val: 0.00000
Step:429, train: 0.03370, val: 0.00000
Step:430, train: 0.03374, val: 0.00000
Step:431, train: 0.03263, val: 0.00000
Step:432, train: 0.03317, val: 0.00000
Step:433, train: 0.03397, val: 0.00000
Step:434, train: 0.03372, val: 0.00000
Step:435, train: 0.03288, val: 0.00000
Step:436, train: 0.03271, val: 0.00000
Step:437, train: 0.03377, val: 0.00000
Step:438, train: 0.03336, val: 0.00000
Step:439, train: 0.03385, val: 0.00000
Step:440, train: 0.03258, val: 0.00000
Step:441, train: 0.03193, val: 0.00000
Step:442, train: 0.03281, val: 0.00000
Step:443, train: 0.03329, val: 0.00000
Step:444, train: 0.03329, val: 0.00000
Step:445, train: 0.03274, val: 0.00000
Step:446, train: 0.03318, val: 0.00000
Step:447, train: 0.03288, val: 0.00000
Step:448, train: 0.03242, val: 0.00000
Step:449, train: 0.03327, val: 0.00000
Step:450, train: 0.03247, val: 0.00000
Step:451, train: 0.03261, val: 0.00000
Step:452, train: 0.03330, val: 0.00000
Step:453, train: 0.03263,

Step:642, train: 0.03030, val: 0.00000
Step:643, train: 0.03061, val: 0.00000
Step:644, train: 0.03004, val: 0.00000
Step:645, train: 0.03029, val: 0.00000
Step:646, train: 0.02977, val: 0.00000
Step:647, train: 0.02963, val: 0.00000
Step:648, train: 0.03067, val: 0.00000
Step:649, train: 0.03049, val: 0.00000
Step:650, train: 0.03015, val: 0.00000
Step:651, train: 0.02936, val: 0.00000
Step:652, train: 0.03079, val: 0.00000
Step:653, train: 0.03012, val: 0.00000
Step:654, train: 0.02987, val: 0.00000
Step:655, train: 0.02977, val: 0.00000
Step:656, train: 0.02908, val: 0.00000
Step:657, train: 0.03054, val: 0.00000
Step:658, train: 0.02994, val: 0.00000
Step:659, train: 0.02978, val: 0.00000
Step:660, train: 0.02907, val: 0.00000
Step:661, train: 0.02968, val: 0.00000
Step:662, train: 0.03004, val: 0.00000
Step:663, train: 0.02979, val: 0.00000
Step:664, train: 0.03001, val: 0.00000
Step:665, train: 0.03018, val: 0.00000
Step:666, train: 0.03021, val: 0.00000
Step:667, train: 0.02993,

Step:856, train: 0.02859, val: 0.00000
Step:857, train: 0.02935, val: 0.00000
Step:858, train: 0.02949, val: 0.00000
Step:859, train: 0.02893, val: 0.00000
Step:860, train: 0.02821, val: 0.00000
Step:861, train: 0.02829, val: 0.00000
Step:862, train: 0.02927, val: 0.00000
Step:863, train: 0.02915, val: 0.00000
Step:864, train: 0.02921, val: 0.00000
Step:865, train: 0.02913, val: 0.00000
Step:866, train: 0.02825, val: 0.00000
Step:867, train: 0.02924, val: 0.00000
Step:868, train: 0.02932, val: 0.00000
Step:869, train: 0.02921, val: 0.00000
Step:870, train: 0.02886, val: 0.00000
Step:871, train: 0.02985, val: 0.00000
Step:872, train: 0.02864, val: 0.00000
Step:873, train: 0.02864, val: 0.00000
Step:874, train: 0.02911, val: 0.00000
Step:875, train: 0.02895, val: 0.00000
Step:876, train: 0.02889, val: 0.00000
Step:877, train: 0.02914, val: 0.00000
Step:878, train: 0.02849, val: 0.00000
Step:879, train: 0.02837, val: 0.00000
Step:880, train: 0.02872, val: 0.00000
Step:881, train: 0.02868,

In [28]:
channel_fidelity(kraus_model, kraus_target).numpy()

0.9363987264160256

### Fit Model, Full Initial and POVM

In [29]:
np.random.seed(45)
random.seed(45)
tf.random.set_seed(45)

spam_model = SPAM(d=d,
#                  use_corr_mat=True,
                  optimizer = tf.optimizers.Adam(learning_rate=0.01))

spam_model.pretrain(targets = [init_ideal(d), povm_ideal(d)],
                    num_iter = 1000,
                    )

spam_model.train(inputs = inputs_spam,
                 targets = targets_spam,
                 num_iter = 1000,
                 N = N_spam
                )

kraus_model = KrausMap(
                       d = d, 
                       rank = rank,
                       spam = spam_model,
                       )

model = ModelQuantumMap(
                        q_map = kraus_model,
                        loss = probs_loss,
                        optimizer = tf.optimizers.Adam(learning_rate=0.01),
                        )

model.train(inputs = inputs_map,
            targets = targets_map,
            #inputs_val = inputs_map,
            #targets_val = targets_map,
            num_iter = 1000,
            N = 1000
            )

  0%|          | 0/1000 [00:00<?, ?it/s]

0 0.03141456756929366
1 0.031166975276331055
2 0.030916752796686753
3 0.030664585189758266
4 0.030410662516107485
5 0.030154870372947576
6 0.029897309762168256
7 0.02963806614652993
8 0.02937681139125036
9 0.02911354777404548
10 0.028848363984930334
11 0.028581186125615797
12 0.028311642995783124
13 0.02803961692382042
14 0.027764981299482495
15 0.027486983029764912
16 0.027205651959188677
17 0.02692028273727074
18 0.02663117095230913
19 0.02633809586244357
20 0.026041232249029295
21 0.025741038233907665
22 0.025437229678864887
23 0.025129361173036005
24 0.02481592867124459
25 0.024497700400147468
26 0.024174843014383517
27 0.0238475057642159
28 0.02351614036917899
29 0.023180251449212225
30 0.022840539469906385
31 0.022496857064196796
32 0.022148732101594598
33 0.021796851142691046
34 0.02144083333224076
35 0.021081066007830183
36 0.020717367075350848
37 0.020349667184994057
38 0.019978074465031033
39 0.01960379939076555
40 0.01922724990401304
41 0.018848439186111638
42 0.018467638016

329 5.0171911397581006e-05
330 4.972300294613762e-05
331 4.927926393583407e-05
332 4.884053721903414e-05
333 4.840655136731511e-05
334 4.797741718609095e-05
335 4.7552950180202404e-05
336 4.7133299607357e-05
337 4.6718466904429164e-05
338 4.6308450247410405e-05
339 4.590314310317884e-05
340 4.550264844520289e-05
341 4.510693836503372e-05
342 4.47159603975153e-05
343 4.432966305229882e-05
344 4.3947968252779906e-05
345 4.357067479103437e-05
346 4.31978144790701e-05
347 4.282929252408123e-05
348 4.246487744882403e-05
349 4.210462003444871e-05
350 4.174837606665283e-05
351 4.139614905291012e-05
352 4.104787816298747e-05
353 4.0703502891573544e-05
354 4.03628945831105e-05
355 4.002606736995506e-05
356 3.969297334277973e-05
357 3.9363580932141966e-05
358 3.9037870249342185e-05
359 3.8715746238194134e-05
360 3.8397203033576305e-05
361 3.808218284583175e-05
362 3.777048623127574e-05
363 3.746222509244208e-05
364 3.7157354497459914e-05
365 3.685576307840385e-05
366 3.6557457017007343e-05
367 3

642 6.724420777728379e-06
643 6.6923411445963125e-06
644 6.660451188356468e-06
645 6.628758571358038e-06
646 6.597248573958805e-06
647 6.565932741816406e-06
648 6.5348142248708376e-06
649 6.503888053642475e-06
650 6.473146002821147e-06
651 6.442593926211538e-06
652 6.412235543188227e-06
653 6.382043778724522e-06
654 6.35204237015655e-06
655 6.322216400535703e-06
656 6.292570418903115e-06
657 6.263109541935188e-06
658 6.233829838821358e-06
659 6.204721186890207e-06
660 6.1757770698171e-06
661 6.147009011386293e-06
662 6.118415697008465e-06
663 6.089998286436506e-06
664 6.061753362598003e-06
665 6.0336802408565e-06
666 6.00576862472574e-06
667 5.978013177743118e-06
668 5.950424507913867e-06
669 5.923001202004672e-06
670 5.89574514066172e-06
671 5.868653169374909e-06
672 5.841724513072929e-06
673 5.814949112932039e-06
674 5.788322121199868e-06
675 5.761853986866802e-06
676 5.735542906198591e-06
677 5.709392989610261e-06
678 5.683399089334031e-06
679 5.657550667923268e-06
680 5.63184751259

968 1.8579458343162898e-06
969 1.8518119663507985e-06
970 1.8457034584384082e-06
971 1.839621812218563e-06
972 1.8335668066680892e-06
973 1.8275323980681039e-06
974 1.8215235081299418e-06
975 1.815540439622523e-06
976 1.809582718232842e-06
977 1.8036500099055784e-06
978 1.79774233471663e-06
979 1.7918592983641616e-06
980 1.7860014951213385e-06
981 1.7801682675015927e-06
982 1.7743555766119624e-06
983 1.7685675771169565e-06
984 1.762803150698112e-06
985 1.757062801292919e-06
986 1.7513469113756401e-06
987 1.745654076808506e-06
988 1.7399857485407442e-06
989 1.7343415888341408e-06
990 1.7287183819911109e-06
991 1.7231187358405438e-06
992 1.7175421477820534e-06
993 1.7119869449486982e-06
994 1.7064550781838883e-06
995 1.7009468204882657e-06
996 1.695461837080333e-06
997 1.6899961944377512e-06
998 1.6845537842567497e-06
999 1.6791343602147956e-06


...

Step:0, train: 0.24126, val: 0.00000
Step:1, train: 0.24300, val: 0.00000
Step:2, train: 0.23962, val: 0.00000
Step:3, train: 0.23311, val: 0.00000
Step:4, train: 0.23143, val: 0.00000
Step:5, train: 0.22973, val: 0.00000
Step:6, train: 0.22703, val: 0.00000
Step:7, train: 0.22764, val: 0.00000
Step:8, train: 0.22678, val: 0.00000
Step:9, train: 0.22420, val: 0.00000
Step:10, train: 0.22282, val: 0.00000
Step:11, train: 0.21860, val: 0.00000
Step:12, train: 0.20843, val: 0.00000
Step:13, train: 0.20940, val: 0.00000
Step:14, train: 0.20997, val: 0.00000
Step:15, train: 0.20742, val: 0.00000
Step:16, train: 0.20100, val: 0.00000
Step:17, train: 0.19784, val: 0.00000
Step:18, train: 0.19782, val: 0.00000
Step:19, train: 0.19780, val: 0.00000
Step:20, train: 0.19164, val: 0.00000
Step:21, train: 0.19509, val: 0.00000
Step:22, train: 0.19549, val: 0.00000
Step:23, train: 0.19264, val: 0.00000
Step:24, train: 0.19120, val: 0.00000
Step:25, train: 0.19157, val: 0.00000
Step:26, train: 0.1840

Step:214, train: 0.03859, val: 0.00000
Step:215, train: 0.03772, val: 0.00000
Step:216, train: 0.03703, val: 0.00000
Step:217, train: 0.03747, val: 0.00000
Step:218, train: 0.03668, val: 0.00000
Step:219, train: 0.03765, val: 0.00000
Step:220, train: 0.03760, val: 0.00000
Step:221, train: 0.03686, val: 0.00000
Step:222, train: 0.03725, val: 0.00000
Step:223, train: 0.03794, val: 0.00000
Step:224, train: 0.03716, val: 0.00000
Step:225, train: 0.03758, val: 0.00000
Step:226, train: 0.03658, val: 0.00000
Step:227, train: 0.03694, val: 0.00000
Step:228, train: 0.03596, val: 0.00000
Step:229, train: 0.03738, val: 0.00000
Step:230, train: 0.03649, val: 0.00000
Step:231, train: 0.03711, val: 0.00000
Step:232, train: 0.03763, val: 0.00000
Step:233, train: 0.03692, val: 0.00000
Step:234, train: 0.03706, val: 0.00000
Step:235, train: 0.03555, val: 0.00000
Step:236, train: 0.03616, val: 0.00000
Step:237, train: 0.03591, val: 0.00000
Step:238, train: 0.03590, val: 0.00000
Step:239, train: 0.03637,

Step:426, train: 0.03038, val: 0.00000
Step:427, train: 0.02988, val: 0.00000
Step:428, train: 0.03059, val: 0.00000
Step:429, train: 0.02942, val: 0.00000
Step:430, train: 0.03068, val: 0.00000
Step:431, train: 0.03048, val: 0.00000
Step:432, train: 0.03035, val: 0.00000
Step:433, train: 0.03064, val: 0.00000
Step:434, train: 0.03038, val: 0.00000
Step:435, train: 0.03022, val: 0.00000
Step:436, train: 0.03039, val: 0.00000
Step:437, train: 0.03148, val: 0.00000
Step:438, train: 0.03062, val: 0.00000
Step:439, train: 0.02970, val: 0.00000
Step:440, train: 0.03003, val: 0.00000
Step:441, train: 0.02944, val: 0.00000
Step:442, train: 0.03067, val: 0.00000
Step:443, train: 0.02953, val: 0.00000
Step:444, train: 0.02950, val: 0.00000
Step:445, train: 0.03014, val: 0.00000
Step:446, train: 0.03007, val: 0.00000
Step:447, train: 0.03035, val: 0.00000
Step:448, train: 0.02984, val: 0.00000
Step:449, train: 0.03059, val: 0.00000
Step:450, train: 0.02958, val: 0.00000
Step:451, train: 0.03048,

Step:640, train: 0.02864, val: 0.00000
Step:641, train: 0.02833, val: 0.00000
Step:642, train: 0.02839, val: 0.00000
Step:643, train: 0.02818, val: 0.00000
Step:644, train: 0.02884, val: 0.00000
Step:645, train: 0.02890, val: 0.00000
Step:646, train: 0.02861, val: 0.00000
Step:647, train: 0.02794, val: 0.00000
Step:648, train: 0.02859, val: 0.00000
Step:649, train: 0.02859, val: 0.00000
Step:650, train: 0.02899, val: 0.00000
Step:651, train: 0.02816, val: 0.00000
Step:652, train: 0.02812, val: 0.00000
Step:653, train: 0.02742, val: 0.00000
Step:654, train: 0.02874, val: 0.00000
Step:655, train: 0.02843, val: 0.00000
Step:656, train: 0.02788, val: 0.00000
Step:657, train: 0.02818, val: 0.00000
Step:658, train: 0.02836, val: 0.00000
Step:659, train: 0.02757, val: 0.00000
Step:660, train: 0.02778, val: 0.00000
Step:661, train: 0.02853, val: 0.00000
Step:662, train: 0.02831, val: 0.00000
Step:663, train: 0.02833, val: 0.00000
Step:664, train: 0.02786, val: 0.00000
Step:665, train: 0.02755,

Step:851, train: 0.02792, val: 0.00000
Step:852, train: 0.02788, val: 0.00000
Step:853, train: 0.02813, val: 0.00000
Step:854, train: 0.02772, val: 0.00000
Step:855, train: 0.02813, val: 0.00000
Step:856, train: 0.02748, val: 0.00000
Step:857, train: 0.02805, val: 0.00000
Step:858, train: 0.02718, val: 0.00000
Step:859, train: 0.02783, val: 0.00000
Step:860, train: 0.02822, val: 0.00000
Step:861, train: 0.02826, val: 0.00000
Step:862, train: 0.02763, val: 0.00000
Step:863, train: 0.02826, val: 0.00000
Step:864, train: 0.02791, val: 0.00000
Step:865, train: 0.02796, val: 0.00000
Step:866, train: 0.02731, val: 0.00000
Step:867, train: 0.02832, val: 0.00000
Step:868, train: 0.02782, val: 0.00000
Step:869, train: 0.02810, val: 0.00000
Step:870, train: 0.02692, val: 0.00000
Step:871, train: 0.02786, val: 0.00000
Step:872, train: 0.02766, val: 0.00000
Step:873, train: 0.02773, val: 0.00000
Step:874, train: 0.02801, val: 0.00000
Step:875, train: 0.02762, val: 0.00000
Step:876, train: 0.02739,

In [30]:
channel_fidelity(kraus_model, kraus_target).numpy()

0.9538934094433418

### Fit Model, Initial and Corruption Matrix

In [31]:
np.random.seed(45)
random.seed(45)
tf.random.set_seed(45)

spam_model = SPAM(d=d,
                  use_corr_mat=True,
                  optimizer = tf.optimizers.Adam(learning_rate=0.01))

spam_model.pretrain(targets = [init_ideal(d), povm_ideal(d)],
                    num_iter = 1000,
                    )

spam_model.train(inputs = inputs_spam,
                 targets = targets_spam,
                 num_iter = 1000,
                 N = N_spam
                )



kraus_model = KrausMap(
                       d = d, 
                       rank = rank,
                       spam = spam_model,
                       )

model = ModelQuantumMap(
                        q_map = kraus_model,
                        loss = probs_loss,
                        optimizer = tf.optimizers.Adam(learning_rate=0.01),
                        )

model.train(inputs = inputs_map,
            targets = targets_map,
            #inputs_val = inputs_map,
            #targets_val = targets_map,
            num_iter = 1000,
            N = 1000
            )

  0%|          | 0/1000 [00:00<?, ?it/s]

0 0.030847143338659015
1 0.030662459515911837
2 0.030478113635720658
3 0.030294378560601604
4 0.03011125446539551
5 0.0299286394951683
6 0.02974637630255854
7 0.029564343058466922
8 0.029382166271182886
9 0.029199761531843116
10 0.029017001016869184
11 0.028834124534501303
12 0.028650785940328858
13 0.028466479769947366
14 0.028281007779332172
15 0.02809361896369352
16 0.027905004486438477
17 0.027714040522829386
18 0.027520756143509136
19 0.027325171289391166
20 0.02712710205235179
21 0.02692724941940549
22 0.02672547779844971
23 0.026520626423110247
24 0.026311699381119276
25 0.026099274888549243
26 0.025883769335170993
27 0.025665682273699086
28 0.02544524040096212
29 0.025222609140702103
30 0.024996647170454134
31 0.024767377827047465
32 0.02453436545109277
33 0.024298003404903394
34 0.024058226701104274
35 0.023815564230044564
36 0.02356915406288541
37 0.023319200323779987
38 0.02306544890680265
39 0.022809350387594865
40 0.022550724877813944
41 0.022290167802329452
42 0.022026576

337 3.461406243846226e-05
338 3.431007868772293e-05
339 3.40096365743444e-05
340 3.371275055427515e-05
341 3.341941368945111e-05
342 3.312958760193579e-05
343 3.284322786155081e-05
344 3.256027993065162e-05
345 3.2280679313774234e-05
346 3.2004357287621025e-05
347 3.173124784660386e-05
348 3.146128804622312e-05
349 3.119440988846168e-05
350 3.093055106659208e-05
351 3.06696514514125e-05
352 3.0411660445555467e-05
353 3.0156524752494703e-05
354 2.9904201138563216e-05
355 2.9654650381537954e-05
356 2.94078432582295e-05
357 2.916375178709954e-05
358 2.8922351681273416e-05
359 2.8683619722250747e-05
360 2.844752731719148e-05
361 2.8214044712288655e-05
362 2.798314104155141e-05
363 2.775477967594176e-05
364 2.7528921813867278e-05
365 2.7305525918907145e-05
366 2.708454976554745e-05
367 2.6865951416733643e-05
368 2.664968946662165e-05
369 2.643571894086855e-05
370 2.6223997221107255e-05
371 2.6014482600826677e-05
372 2.5807135281548766e-05
373 2.5601918581425948e-05
374 2.5398799120684647e-0

649 4.813118927080195e-06
650 4.790375069785903e-06
651 4.7677690586835265e-06
652 4.745299871408363e-06
653 4.722966506364772e-06
654 4.7007679810144976e-06
655 4.678703329774621e-06
656 4.6567716016469145e-06
657 4.634971857709996e-06
658 4.61330316860637e-06
659 4.591764612150547e-06
660 4.570355271171402e-06
661 4.54907423168667e-06
662 4.527920581481385e-06
663 4.506893409139213e-06
664 4.4859918035468605e-06
665 4.465214853866859e-06
666 4.444561649942291e-06
667 4.424031283084067e-06
668 4.403622847159602e-06
669 4.38333543990124e-06
670 4.363168164333473e-06
671 4.343120130222465e-06
672 4.3231904554524225e-06
673 4.3033782672445005e-06
674 4.283682703144606e-06
675 4.26410291172888e-06
676 4.244638052991568e-06
677 4.2252872984058985e-06
678 4.2060498306661835e-06
679 4.186924843142382e-06
680 4.1679115390936005e-06
681 4.149009130704429e-06
682 4.130216838010251e-06
683 4.111533887790694e-06
684 4.092959512504314e-06
685 4.074492949330304e-06
686 4.056133439382408e-06
687 4.0

972 1.3614625676577853e-06
973 1.3570032567705427e-06
974 1.352562388784008e-06
975 1.348139874152169e-06
976 1.3437356248805604e-06
977 1.3393495541853374e-06
978 1.3349815761217576e-06
979 1.3306316052082805e-06
980 1.3262995560699416e-06
981 1.321985343124364e-06
982 1.317688880328323e-06
983 1.3134100810013127e-06
984 1.3091488577342158e-06
985 1.3049051223869263e-06
986 1.3006787861713385e-06
987 1.2964697598092613e-06
988 1.2922779537532967e-06
989 1.288103278449292e-06
990 1.2839456446217356e-06
991 1.279804963558158e-06
992 1.2756811473731555e-06
993 1.2715741092316623e-06
994 1.2674837635177417e-06
995 1.263410025937082e-06
996 1.2593528135501987e-06
997 1.255312044735514e-06
998 1.2512876390894938e-06
999 1.2472795172747822e-06


...

Step:0, train: 0.23965, val: 0.00000
Step:1, train: 0.24307, val: 0.00000
Step:2, train: 0.24606, val: 0.00000
Step:3, train: 0.23835, val: 0.00000
Step:4, train: 0.23409, val: 0.00000
Step:5, train: 0.23239, val: 0.00000
Step:6, train: 0.23294, val: 0.00000
Step:7, train: 0.22695, val: 0.00000
Step:8, train: 0.22352, val: 0.00000
Step:9, train: 0.22146, val: 0.00000
Step:10, train: 0.22175, val: 0.00000
Step:11, train: 0.22454, val: 0.00000
Step:12, train: 0.21165, val: 0.00000
Step:13, train: 0.20614, val: 0.00000
Step:14, train: 0.21157, val: 0.00000
Step:15, train: 0.21093, val: 0.00000
Step:16, train: 0.20841, val: 0.00000
Step:17, train: 0.20351, val: 0.00000
Step:18, train: 0.19598, val: 0.00000
Step:19, train: 0.19793, val: 0.00000
Step:20, train: 0.19654, val: 0.00000
Step:21, train: 0.19715, val: 0.00000
Step:22, train: 0.19431, val: 0.00000
Step:23, train: 0.19518, val: 0.00000
Step:24, train: 0.19315, val: 0.00000
Step:25, train: 0.18854, val: 0.00000
Step:26, train: 0.1952

Step:214, train: 0.03687, val: 0.00000
Step:215, train: 0.03757, val: 0.00000
Step:216, train: 0.03697, val: 0.00000
Step:217, train: 0.03689, val: 0.00000
Step:218, train: 0.03633, val: 0.00000
Step:219, train: 0.03581, val: 0.00000
Step:220, train: 0.03683, val: 0.00000
Step:221, train: 0.03713, val: 0.00000
Step:222, train: 0.03578, val: 0.00000
Step:223, train: 0.03686, val: 0.00000
Step:224, train: 0.03676, val: 0.00000
Step:225, train: 0.03670, val: 0.00000
Step:226, train: 0.03605, val: 0.00000
Step:227, train: 0.03637, val: 0.00000
Step:228, train: 0.03557, val: 0.00000
Step:229, train: 0.03605, val: 0.00000
Step:230, train: 0.03652, val: 0.00000
Step:231, train: 0.03497, val: 0.00000
Step:232, train: 0.03583, val: 0.00000
Step:233, train: 0.03642, val: 0.00000
Step:234, train: 0.03615, val: 0.00000
Step:235, train: 0.03651, val: 0.00000
Step:236, train: 0.03560, val: 0.00000
Step:237, train: 0.03580, val: 0.00000
Step:238, train: 0.03518, val: 0.00000
Step:239, train: 0.03525,

Step:426, train: 0.02893, val: 0.00000
Step:427, train: 0.02903, val: 0.00000
Step:428, train: 0.02818, val: 0.00000
Step:429, train: 0.02915, val: 0.00000
Step:430, train: 0.02833, val: 0.00000
Step:431, train: 0.02894, val: 0.00000
Step:432, train: 0.02901, val: 0.00000
Step:433, train: 0.02865, val: 0.00000
Step:434, train: 0.02854, val: 0.00000
Step:435, train: 0.02882, val: 0.00000
Step:436, train: 0.02792, val: 0.00000
Step:437, train: 0.02868, val: 0.00000
Step:438, train: 0.02951, val: 0.00000
Step:439, train: 0.02863, val: 0.00000
Step:440, train: 0.02799, val: 0.00000
Step:441, train: 0.02844, val: 0.00000
Step:442, train: 0.02803, val: 0.00000
Step:443, train: 0.02826, val: 0.00000
Step:444, train: 0.02789, val: 0.00000
Step:445, train: 0.02793, val: 0.00000
Step:446, train: 0.02780, val: 0.00000
Step:447, train: 0.02792, val: 0.00000
Step:448, train: 0.02833, val: 0.00000
Step:449, train: 0.02835, val: 0.00000
Step:450, train: 0.02883, val: 0.00000
Step:451, train: 0.02747,

Step:639, train: 0.02622, val: 0.00000
Step:640, train: 0.02684, val: 0.00000
Step:641, train: 0.02690, val: 0.00000
Step:642, train: 0.02646, val: 0.00000
Step:643, train: 0.02636, val: 0.00000
Step:644, train: 0.02625, val: 0.00000
Step:645, train: 0.02705, val: 0.00000
Step:646, train: 0.02711, val: 0.00000
Step:647, train: 0.02680, val: 0.00000
Step:648, train: 0.02637, val: 0.00000
Step:649, train: 0.02675, val: 0.00000
Step:650, train: 0.02680, val: 0.00000
Step:651, train: 0.02703, val: 0.00000
Step:652, train: 0.02624, val: 0.00000
Step:653, train: 0.02638, val: 0.00000
Step:654, train: 0.02586, val: 0.00000
Step:655, train: 0.02712, val: 0.00000
Step:656, train: 0.02698, val: 0.00000
Step:657, train: 0.02629, val: 0.00000
Step:658, train: 0.02643, val: 0.00000
Step:659, train: 0.02669, val: 0.00000
Step:660, train: 0.02623, val: 0.00000
Step:661, train: 0.02609, val: 0.00000
Step:662, train: 0.02669, val: 0.00000
Step:663, train: 0.02686, val: 0.00000
Step:664, train: 0.02655,

Step:851, train: 0.02582, val: 0.00000
Step:852, train: 0.02628, val: 0.00000
Step:853, train: 0.02626, val: 0.00000
Step:854, train: 0.02636, val: 0.00000
Step:855, train: 0.02611, val: 0.00000
Step:856, train: 0.02599, val: 0.00000
Step:857, train: 0.02572, val: 0.00000
Step:858, train: 0.02635, val: 0.00000
Step:859, train: 0.02568, val: 0.00000
Step:860, train: 0.02629, val: 0.00000
Step:861, train: 0.02617, val: 0.00000
Step:862, train: 0.02652, val: 0.00000
Step:863, train: 0.02554, val: 0.00000
Step:864, train: 0.02652, val: 0.00000
Step:865, train: 0.02632, val: 0.00000
Step:866, train: 0.02627, val: 0.00000
Step:867, train: 0.02564, val: 0.00000
Step:868, train: 0.02647, val: 0.00000
Step:869, train: 0.02574, val: 0.00000
Step:870, train: 0.02611, val: 0.00000
Step:871, train: 0.02500, val: 0.00000
Step:872, train: 0.02597, val: 0.00000
Step:873, train: 0.02595, val: 0.00000
Step:874, train: 0.02596, val: 0.00000
Step:875, train: 0.02614, val: 0.00000
Step:876, train: 0.02588,

In [32]:
channel_fidelity(kraus_model, kraus_target).numpy()

0.9839532579597369